In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import pykitti
from scipy.spatial.transform import Rotation as R

import vedo
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import nbconvert

%autosave 90
%matplotlib notebook

2023-08-15 15:29:02.505196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Autosaving every 90 seconds


In [2]:
# A = np.random.randn(1_000,3)

In [18]:
# Load KITTI sample dataset
# Locate dataset through base directory 
basedir = "/home/danielchoate/Documents"
date = '2011_09_26'

# urban dataset used in 3D-ICET paper 
drive = '0005' #life in the big city
# drive = '0091' #Matt's favorite- don't use for testing though, we trained here!

# Select the index to view 
idx = 0 #frame index in sequence
skip = 1

dataset = pykitti.raw(basedir, date, drive)

velo1 = dataset.get_velo(idx) # Each scan is a Nx4 array of [x,y,z,reflectance]
c1 = velo1[:,:3]
velo2 = dataset.get_velo(idx+skip) # Each scan is a Nx4 array of [x,y,z,reflectance]
c2 = velo2[:,:3]
# c1 = c1[c1[:,2] > -1.5] #ignore ground plane
# c2 = c2[c2[:,2] > -1.5] #ignore ground plane
c1 = c1[c1[:,2] > -2.] #ignore reflections
c2 = c2[c2[:,2] > -2.] #ignore reflections

poses0 = dataset.oxts[idx] #<- ID of 1st scan
poses1 = dataset.oxts[idx+1] #<- ID of 2nd scan
dt = skip*0.10 #mean time between lidar samples
OXTS_ground_truth = tf.constant([poses1.packet.vf*dt, -poses1.packet.vl*dt, poses1.packet.vu*dt, poses1.packet.wf*dt, poses1.packet.wl*dt, poses1.packet.wu*dt])

In [21]:
# Define the plot 
# N determines the window size
# Axes determines the axes listed on projection
# bg determines the color scheme of the plane background 
# Must make the scene interactive 
plt = Plotter(N=1, axes=4, bg=(1,1,1), interactive = True)
disp = []

# ***Not entirely sure what following two commands control
disp.append(Points(c1, c = 'red', r = 4))
# disp.append(Points(c2, c = 'blue', r = 3))

# *** Further explore rotation matrices
Rotation_matrix = R.from_euler('xyz', -OXTS_ground_truth[3:]).as_matrix()
# c2_in_c1_frame = c2 + OXTS_ground_truth[:3] #translation only
c2_in_c1_frame = c2 @ Rotation_matrix + OXTS_ground_truth[:3]
disp.append(Points(c2_in_c1_frame, c = 'blue', r = 3))

disp.append(Points([[0.,0.,0.]], c ='purple',r=10))

# Show plot and create a title for the plot 
plt.show(disp, "test")

# ViewInteractiveWidget(plt.window) 
# ^^^ only works once, will freeze up notebook
#     you will have to kernel (button at top of page) -> restart & clear output to get things working again

In [7]:
print(OXTS_ground_truth)

tf.Tensor([ 0.34886387 -0.00547039  0.00110176 -0.00041659  0.00442824  0.01399535], shape=(6,), dtype=float32)


In [8]:
# Rotation_matrix = R.from_euler('xyz', OXTS_ground_truth[3:]).as_matrix()
# print(Rotation_matrix)